In [ ]:
!pip uninstall -y torch torchvision torchaudio accelerate transformers sentence_transformers
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 accelerate==0.25.0 transformers==4.36.0 sentence_transformers==2.7.0
!pip install eland elasticsearch

In [ ]:
!pip install

In [ ]:
from pathlib import Path
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient

<ipython-input-4-ed653d9ea084>:5: DeprecationWarning: Importing from the 'elasticsearch.client' module is deprecated. Instead use 'elasticsearch' module for importing the client.
  from elasticsearch.client import MlClient


In [ ]:
import getpass
es_cloud_id = getpass.getpass("Enter Elastic Cloud ID: ")

api_id = getpass.getpass("Enter Elastic API ID: ")

api_key = getpass.getpass("Enter Elastic API KEY: ")

# Create the client instance
client = Elasticsearch(cloud_id=es_cloud_id, api_key=(api_id, api_key))

client.info()

Enter Elastic Cloud ID: ··········
Enter Elastic API ID: ··········
Enter Elastic API KEY: ··········


ObjectApiResponse({'name': 'instance-0000000001', 'cluster_name': '5089524370ab4e1396e3270ca581c541', 'cluster_uuid': 'LFC-6cmGSsqshnnawgItBg', 'version': {'number': '9.0.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '73f7594ea00db50aa7e941e151a5b3985f01e364', 'build_date': '2025-04-30T10:07:41.393025990Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [ ]:
hf_model_id='intfloat/multilingual-e5-base'
tm = TransformerModel(model_id=hf_model_id, task_type="text_embedding")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

...

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
es_model_id = tm.elasticsearch_model_id()
es_model_id

'intfloat__multilingual-e5-base'

In [ ]:
tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

In [ ]:
ptm = PyTorchModel(client, es_model_id)
ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config)

  0%|          | 0/1059 [00:00<?, ? parts/s]

In [ ]:
client

<Elasticsearch(['https://5089524370ab4e1396e3270ca581c541.us-central1.gcp.cloud.es.io:443'])>

In [ ]:
m = MlClient.get_trained_models(client, model_id=es_model_id)

In [ ]:
MlClient.start_trained_model_deployment(client, model_id=es_model_id)

In [ ]:
stats = MlClient.get_trained_models_stats(client, model_id=es_model_id)
stats.body['trained_model_stats'][0]['model_id']

'intfloat__multilingual-e5-base'

In [ ]:
docs = [
    {
        "text_field": "안녕하세요. 테스트 입니다."
    }
 ]

result = MlClient.infer_trained_model(client, model_id=es_model_id, docs=docs)

doc_0_vector = result['inference_results'][0]['predicted_value']
print(len(doc_0_vector))

768


In [ ]:
len(doc_0_vector)

768

**Reasoning**:
Import necessary libraries and connect to the Elasticsearch cluster using provided credentials.

